In [1]:
import os
import pandas as pd
import calendar
import datetime
import monpa
from monpa import utils
import re
import numpy as np
import copy
# from wakepy import set_keepawake, unset_keepawake

# set_keepawake(keep_screen_awake=True)

os.chdir('/Users/arthur/Desktop/factor_investment')

/opt/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [25]:
# 統深、數商上課內容
# 期中報告
# 寫常用函式庫

In [26]:
# 關聯分析 git
#v 平均分數
#v -1, 0, 1
# 成本 3/1000, 1.425/1000
# 同成本非同張數
# word embedding
# git https://chwang12341.medium.com/machine-learning-關聯分析-apriori演算法-詳細解說啤酒與尿布的背後原理-python實作-scikit-learn一步一步教學-76b7778f8f34

# 加上之前的base model也跑出結果
# 這禮拜先word embedding+簡報（過去加現在的結果）

In [27]:
# 少一點因子會比較好嗎
# 沒有考慮IR
# 精進輿情分析
# 標準化？

In [2]:
from alphalens.tears import (create_returns_tear_sheet,
                      create_information_tear_sheet,
                      create_turnover_tear_sheet,
                      create_summary_tear_sheet,
                      create_full_tear_sheet,
                      create_event_returns_tear_sheet,
                      create_event_study_tear_sheet)

from alphalens.utils import get_clean_factor_and_forward_returns

In [3]:
# 我們從stopwords_zh.txt這個檔案中匯入繁體中文的停用詞
with open('./202202_Intern/stopwords_zh.txt', 'r') as file:
    stopwords = file.read().splitlines()
file.close()

with open('./202202_Intern/NTUSD_negative_unicode.txt', 'r') as file:
    nega_words = file.read().splitlines()
file.close()

with open('./202202_Intern/NTUSD_positive_unicode.txt', 'r') as file:
    posi_words = file.read().splitlines()
file.close()

# 這個function用來將字串以  **正則化**  處理去除中文字元以外的字元
def clearSentence(sentence):
    return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

def splitSentence(content):
    sentence_list = utils.short_sentence(content)
    tokenStr = []
    for sentence in sentence_list:
        sentence = clearSentence(sentence)
        tokens = monpa.cut(sentence)
        tokenStr += tokens
    return tokenStr


def calSen(content):
    sentence_list = utils.short_sentence(content)
    tokenStr = []
    for sentence in sentence_list:
        sentence = clearSentence(sentence)
        tokens = monpa.cut(sentence)
        tokenStr += tokens
        
    senScore = 0
    for i in tokenStr:
        if i in posi_words:
            senScore += 1
        elif i in nega_words:
            senScore -= 1
    return senScore

def find_key(dic, value):
    for keys, values in dic.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
        if value == values:
            return(keys)
            break

def trend(value):
    if value > 0:
        return 1
    elif value < 0:
        return -1
    else:
        return 0

# coid_dic

In [30]:
col_list = ["證券代碼", "簡稱", "年月日", "收盤價(元)"]
price = pd.read_csv('./data/TEJ_StockPrice_20052022.csv', encoding='big5', encoding_errors='ignore', usecols=col_list)
price["證券代碼"] = [str(x).strip() for x in price["證券代碼"]]
price["簡稱"] = [x.strip() for x in price["簡稱"]]
price.年月日 = pd.to_datetime(price.年月日).dt.date
price = price[price.年月日 > datetime.date(2012,12,31)].reset_index(drop=True)
price = price[price.年月日 < datetime.date(2022,1,1)].reset_index(drop=True)

/var/folders/w8/jwxq4stj29506b1pf67xpb000000gq/T/ipykernel_3143/449339189.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  price = pd.read_csv('./data/TEJ_StockPrice_20052022.csv', encoding='big5', encoding_errors='ignore', usecols=col_list)


In [31]:
# price[price['證券代碼']=='1311']

In [32]:
coid_stock = price[["證券代碼", "簡稱"]].drop_duplicates().reset_index(drop=True)

In [33]:
price.證券代碼.unique()

array(['15', '50', '51', ..., 'Y9999', 'Z50TR', 'ZGT50'], dtype=object)

In [34]:
coid_stock.loc[coid_stock.證券代碼==1311, '簡稱'].values

array([], dtype=object)

In [35]:
# candidatePoolDF = pd.read_excel('./計算溢酬/CTBCdata/10個因子原始資料.xlsx','125檔', dtype=str,index_col=0,header=1)
# candidatePool=list(candidatePoolDF.index)
candidatePoolDF = pd.read_csv('./data/StockValidation.csv')
candidatePool=list(candidatePoolDF['stock_id'])
candidatePool=[str(x) for x in candidatePool]

price = price.loc[price.證券代碼.isin(candidatePool), :].reset_index(drop=True)
candidatePool = np.unique(price.證券代碼.values)

coid_dic = {}

for i in candidatePool:
    # candidatePool中的1311只在2005~2008，不是我們要的
    try:
#         print(i)
        coid = i
        name = coid_stock.loc[coid_stock.證券代碼 == i, "簡稱"].values[0]
        coid_dic[coid] = name
    except:
        continue
#     print(coid_dic[i])

In [36]:
print(len(coid_dic))
coid_dic

1119


{'1101': '台泥',
 '1102': '亞泥',
 '1103': '嘉泥',
 '1104': '環泥',
 '1108': '幸福',
 '1109': '信大',
 '1110': '東泥',
 '1201': '味全',
 '1203': '味王',
 '1210': '大成',
 '1215': '卜蜂',
 '1216': '統一',
 '1217': '愛之味',
 '1218': '泰山',
 '1219': '福壽',
 '1225': '福懋油',
 '1227': '佳格',
 '1229': '聯華',
 '1231': '聯華食',
 '1232': '大統益',
 '1233': '天仁',
 '1234': '黑松',
 '1256': '鮮活果汁-KY',
 '1259': '安心',
 '1262': '綠悅-KY',
 '1264': '德麥',
 '1301': '台塑',
 '1303': '南亞',
 '1304': '台聚',
 '1305': '華夏',
 '1307': '三芳',
 '1308': '亞聚',
 '1309': '台達化',
 '1310': '台苯',
 '1312': '國喬',
 '1313': '聯成',
 '1314': '中石化',
 '1315': '達新',
 '1319': '東陽',
 '1321': '大洋',
 '1325': '琱j',
 '1326': '台化',
 '1333': '恩得利',
 '1337': '再生-KY',
 '1338': '廣華-KY',
 '1339': '昭輝',
 '1340': '勝悅-KY',
 '1402': '遠東新',
 '1409': '新纖',
 '1416': '廣豐',
 '1419': '新紡',
 '1434': '福懋',
 '1436': '華友聯',
 '1437': '勤益控',
 '1440': '南紡',
 '1442': '名軒',
 '1444': '力麗',
 '1447': '力鵬',
 '1451': '年興',
 '1455': '集盛',
 '1459': '聯發',
 '1460': '宏遠',
 '1464': '得力',
 '1471': '首利',
 '1473': '台南'

In [38]:
print(price.shape)
prices = pd.DataFrame(columns = candidatePool)
for i in price.index:
    coid = price.loc[i, "證券代碼"]
    date = price.loc[i, "年月日"]
    close = price.loc[i, "收盤價(元)"]
    prices.loc[date, coid] = close
    if (i % 10000) == 0:
        print(i//10000, end=' ')

(2180235, 4)
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 

In [40]:
for i in pd.date_range(start='2013-01-01', end='2021-12-31').date:
    if i not in prices.index.values:
        print(i)
        prices.loc[i, :] = [np.nan]*1119

prices = prices.sort_index()
prices = prices.fillna(method='ffill')

2013-01-01
2013-01-05
2013-01-06
2013-01-12
2013-01-13
2013-01-19
2013-01-20
2013-01-26
2013-01-27
2013-02-02
2013-02-03
2013-02-07
2013-02-08
2013-02-09
2013-02-10
2013-02-11
2013-02-12
2013-02-13
2013-02-14
2013-02-15
2013-02-16
2013-02-17
2013-02-24
2013-02-28
2013-03-02
2013-03-03
2013-03-09
2013-03-10
2013-03-16
2013-03-17
2013-03-23
2013-03-24
2013-03-30
2013-03-31
2013-04-04
2013-04-05
2013-04-06
2013-04-07
2013-04-13
2013-04-14
2013-04-20
2013-04-21
2013-04-27
2013-04-28
2013-05-01
2013-05-04
2013-05-05
2013-05-11
2013-05-12
2013-05-18
2013-05-19
2013-05-25
2013-05-26
2013-06-01
2013-06-02
2013-06-08
2013-06-09
2013-06-12
2013-06-15
2013-06-16
2013-06-22
2013-06-23
2013-06-29
2013-06-30
2013-07-06
2013-07-07
2013-07-13
2013-07-14
2013-07-20
2013-07-21
2013-07-27
2013-07-28
2013-08-03
2013-08-04
2013-08-10
2013-08-11
2013-08-17
2013-08-18
2013-08-21
2013-08-24
2013-08-25
2013-08-31
2013-09-01
2013-09-07
2013-09-08
2013-09-15
2013-09-19
2013-09-20
2013-09-21
2013-09-22
2013-09-28

2019-03-16
2019-03-17
2019-03-23
2019-03-24
2019-03-30
2019-03-31
2019-04-04
2019-04-05
2019-04-06
2019-04-07
2019-04-13
2019-04-14
2019-04-20
2019-04-21
2019-04-27
2019-04-28
2019-05-01
2019-05-04
2019-05-05
2019-05-11
2019-05-12
2019-05-18
2019-05-19
2019-05-25
2019-05-26
2019-06-01
2019-06-02
2019-06-07
2019-06-08
2019-06-09
2019-06-15
2019-06-16
2019-06-22
2019-06-23
2019-06-29
2019-06-30
2019-07-06
2019-07-07
2019-07-13
2019-07-14
2019-07-20
2019-07-21
2019-07-27
2019-07-28
2019-08-03
2019-08-04
2019-08-09
2019-08-10
2019-08-11
2019-08-17
2019-08-18
2019-08-24
2019-08-25
2019-08-31
2019-09-01
2019-09-07
2019-09-08
2019-09-13
2019-09-14
2019-09-15
2019-09-21
2019-09-22
2019-09-28
2019-09-29
2019-09-30
2019-10-05
2019-10-06
2019-10-10
2019-10-11
2019-10-12
2019-10-13
2019-10-19
2019-10-20
2019-10-26
2019-10-27
2019-11-02
2019-11-03
2019-11-09
2019-11-10
2019-11-16
2019-11-17
2019-11-23
2019-11-24
2019-11-30
2019-12-01
2019-12-07
2019-12-08
2019-12-14
2019-12-15
2019-12-21
2019-12-22

In [46]:
prices_trend = copy.copy(prices)
for i in prices.columns:
    prices_trend[i] = prices[i].rolling(window=2).apply( lambda x: 1 if ((x[1]-x[0])>0) else( -1 if ((x[1]-x[0])<0) else 0 ) )

In [47]:
prices

,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9940,9941,9942,9943,9945,9946,9951,9955,9957,9958
2013-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-02,19.4604,22.9890,9.8299,9.7465,4.8295,7.9000,12.9092,33.6474,14.3701,11.7970,...,18.8132,66.8,31.7298,21.3076,24.8034,9.0194,29.4123,28.6743,9.00,10.0287
2013-01-03,19.3856,23.0197,9.8654,9.8672,4.8942,7.8631,13.0018,33.6037,14.4766,11.7532,...,18.7035,68.0,31.6335,20.9988,26.5045,8.9721,29.3428,28.8221,9.10,9.9173
2013-01-04,19.4604,22.8355,9.7945,10.0483,4.9806,7.8631,12.7704,33.4728,14.3701,11.7970,...,18.7912,68.3,31.7619,21.0245,26.1484,9.0431,28.9256,28.8221,9.10,9.7687
2013-01-05,19.4604,22.8355,9.7945,10.0483,4.9806,7.8631,12.7704,33.4728,14.3701,11.7970,...,18.7912,68.3,31.7619,21.0245,26.1484,9.0431,28.9256,28.8221,9.10,9.7687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,48.1000,44.1500,20.5000,21.5500,11.8000,20.1500,20.0000,22.0000,32.5500,51.2601,...,35.5500,173.0,91.9000,60.7000,63.0000,15.0000,78.7000,17.2500,11.35,110.5000
2021-12-28,48.1500,44.2000,20.7000,21.6000,11.7500,20.3000,20.1000,22.1500,32.8000,52.1355,...,35.1500,174.0,91.9000,60.8000,63.0000,15.0000,78.3000,17.2000,11.30,110.5000
2021-12-29,48.1500,44.2500,20.8500,21.5500,11.9000,20.4000,20.3500,22.3500,32.8000,51.9410,...,35.2500,174.5,96.9000,60.5000,63.5000,15.1000,78.8000,17.2000,11.40,111.0000
2021-12-30,48.0000,44.3000,20.7500,21.6000,11.9000,20.8000,20.2500,22.2500,32.5500,51.9410,...,35.0500,175.0,98.1000,60.4000,63.8000,15.1000,78.9000,17.2500,11.35,110.5000


In [57]:
prices_trend = prices_trend.iloc[2:, :]
prices_trend.index = pd.date_range(start='2013-01-02', end='2021-12-30').date
prices_trend

,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9940,9941,9942,9943,9945,9946,9951,9955,9957,9958
2013-01-02,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,...,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0
2013-01-03,1.0,-1.0,-1.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,1.0,...,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,0.0,0.0,-1.0
2013-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-06,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-26,1.0,-1.0,0.0,-1.0,1.0,-1.0,0.0,1.0,1.0,0.0,...,-1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,0.0,-1.0,0.0
2021-12-27,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0
2021-12-28,0.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,0.0,-1.0,...,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,0.0,1.0,1.0
2021-12-29,-1.0,1.0,-1.0,1.0,0.0,1.0,-1.0,-1.0,-1.0,0.0,...,-1.0,1.0,1.0,-1.0,1.0,0.0,1.0,1.0,-1.0,-1.0


# OpenIE

In [5]:
from gensim.test.utils import common_texts
import numpy as np
import pandas as pd

# from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
common_texts = [['我', '喜歡', '吃', '水果'], ['他', '愛', '吃', '蔬菜']]
common_texts

[['我', '喜歡', '吃', '水果'], ['他', '愛', '吃', '蔬菜']]

In [19]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [18]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
documents

[TaggedDocument(words=['human', 'interface', 'computer'], tags=[0]),
 TaggedDocument(words=['survey', 'user', 'computer', 'system', 'response', 'time'], tags=[1]),
 TaggedDocument(words=['eps', 'user', 'interface', 'system'], tags=[2]),
 TaggedDocument(words=['system', 'human', 'system', 'eps'], tags=[3]),
 TaggedDocument(words=['user', 'response', 'time'], tags=[4]),
 TaggedDocument(words=['trees'], tags=[5]),
 TaggedDocument(words=['graph', 'trees'], tags=[6]),
 TaggedDocument(words=['graph', 'minors', 'trees'], tags=[7]),
 TaggedDocument(words=['graph', 'minors', 'survey'], tags=[8])]

In [3]:
common_texts[1]

['他', '愛', '吃', '蔬菜']

In [4]:
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [5]:
model = Word2Vec.load("word2vec.model")
model.train([["hello", "world"]], total_examples=1, epochs=1)

In [ ]:
news = pd.read_csv('./202202_Intern/data/201311_cnyesnews.csv')[['date', 'title', 'content']]
news['doc'] = news['title']+news['content']
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate([splitSentence(news.doc.values[0])])]
model = Doc2Vec(documents, vector_size=10, window=5, min_count=1, workers=10)

# for i in news.doc.values:
#     documents = [TaggedDocument(doc, [i]) for i, doc in enumerate([splitSentence(i)])]
#     model = Doc2Vec(documents, vector_size=10, window=5, min_count=1, workers=10)



for i in range(2013, 2022):
    for j in range(1, 13):
        for k in range(1, 3):
            news = pd.read_csv('./202202_Intern/data/'+str(i)+str(j)+str(k)+'_cnyesnews.csv')[['date', 'title', 'content']]
            news['doc'] = news['title']+news['content']
            news.doc = news.doc.fillna('')
            print(i, j, k)
            for m in range(len(news.doc.values)):
                print(m, end=' ')
                documents = [TaggedDocument(doc, [i]) for i, doc in enumerate([splitSentence(news.doc.values[m])])]
                model.train(documents, total_examples=1, epochs=1)
                if m == (len(news.doc.values)-1):
                    model.save("doc2vec" + str(i) + str(j) + str(k) + ".model")

2013 1 1
0 

06/13/2022 09:42:13 - WARNING - gensim.models.word2vec - Effective 'alpha' higher than previous training cycles


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

...

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [ ]:
for i in range(2013, 2022):
    for j in range(1, 13):
        for k in range(1, 3):
            news = pd.read_csv('./202202_Intern/data/'+str(i)+str(j)+str(k)+'_cnyesnews.csv')[['date', 'title', 'content']]
            news['doc'] = news['title']+news['content']
            news.doc = news.doc.fillna('')
            print(i, j, k)
            for m in range(len(news.doc.values)):
                news.doc[m] = splitSentence(news.doc[m])
                
            documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(news.doc.values)]

In [25]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [33]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
documents
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)
model.train(documents, total_examples=1, epochs=1)

06/10/2022 10:56:55 - WARNING - gensim.models.word2vec - Effective 'alpha' higher than previous training cycles
06/10/2022 10:56:55 - WARNING - gensim.models.word2vec - EPOCH 0: supplied example count (9) did not equal expected count (1)


In [39]:
documents

[TaggedDocument(words=['期交所', '訂', '大', '目標', '拚', '期貨', '商品', '與', '國外', '交易所', '相互', '掛牌', '台灣', '期交所', '董事長', '范志強', '今日', '表示', '今年', '期交所', '訂出', '大', '努力', '目標', '其中', '為', '利於', '在', '變化', '快速', '的', '國際', '衍生', '性', '商品', '市場', '中', '競爭', '期交所', '將', '積極', '展開', '國際', '交流力', '拚', '期貨', '商品', '與', '國外', '交易', '所', '互相', '掛牌', '而', '非', '一般', '僅', '止於', '簽訂', '備忘錄', '期交所', '今天', '舉行', '年終', '記者會', '除了', '去年', '期交所', '獲利', '繳出', '每', '股', '稅', '後', '盈餘', '達', '元居', '證券', '周邊', '單位', '之', '冠', '的', '成績', '外', '展望', '今年', '更', '訂出', '一', '全面', '提升', '風控', '及', '監理', '能力', '二', '持續', '推出', '新', '商品', '或', '調整', '制度', '三', '推動', '國際', '合作', '等', '大', '目標', '范志強', '表示', '有鑑於', '近年', '來', '國外', '金融', '市場', '震盪', '加劇', '期交所', '與', '期商公會', '共同', '研議', '強化', '期貨', '市場', '風險', '控管', '機制', '相關', '措施', '期交所', '將', '於', '今年', '建置', '期貨商', '申報', '交易人', '權益數', '完成', '後', '即可', '進行', '線', '上', '風控', '即時', '管理', '交易人', '之', '權益數', '及', '保證金', '屆時', '將', '與', '現行', '每', '日', '執行', '之', '線', '上', '監

In [42]:
model.dv[0]

array([-0.25534233, -0.39239752, -0.3854711 ,  0.5718958 ,  0.15208946,
        0.01703915, -0.35293788, -0.1012956 , -0.65739   , -0.03939687],
      dtype=float32)